In [1]:
# Блок 1: Импорт библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold, ShuffleSplit
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.compose import ColumnTransformer


In [2]:
# Блок 2: Загрузка данных
data = pd.read_excel('yvc.xlsx')
df = pd.DataFrame(data)

print(df.columns.tolist())

['ув', 'ус', 'х1', 'х2', 'х3', 'х4', 'х5', 'х6', 'х7', 'х8', 'х9', 'х10']


In [3]:
# Блок 3: Определение входных и выходных данных
X = df[['х1', 'х2', 'х3', 'х4', 'х5', 'х6', 'х7', 'х8', 'х9', 'х10']]
y = df['ус']

print(X, y)

      х1    х2    х3         х4    х5       х6   х7    х8    х9        х10
0   9.20  11.1  4.30  91.631206   2.0   9.7000  256  34.0  0.90  51.232390
1   9.40  10.5  4.15  88.652482   8.0   9.9000  256  31.0  0.50  50.831217
2   9.10  10.9  4.10  91.835206   2.0   9.6000  256  31.0  0.57  50.831217
3   9.50  11.2  4.25  90.229885   2.0  10.0000  294  31.0  0.70  50.230144
4   9.00  10.8  4.40  92.307692   2.0   9.5000  313  33.0  0.66  50.229696
..   ...   ...   ...        ...   ...      ...  ...   ...   ...        ...
85  9.80  10.5  4.25  91.050725   2.0  10.3300  341  28.0  0.95  49.144037
86  9.95  10.0  3.90  92.152778   2.0   9.9500  256  32.0  0.60  50.969100
87  9.40  11.2  4.25  92.912281   3.0   9.9000  277  30.0  0.70  50.346415
88  9.70  12.0  4.40  90.694444   0.1  10.2378  210  33.2  0.82  51.989188
89  8.90  11.9  4.40  92.061856  16.0   9.4000  349  29.0  0.60  49.195726

[90 rows x 10 columns] 0     27
1     27
2     27
3     24
4     29
      ..
85    28
86    29
87  

In [4]:
# Блок 4: Разделение данных на обучающую, тестовую выборки

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=142)

In [5]:
# Нормализация данных: приведение всех признаков к одному масштабу
scaler = StandardScaler()  # Создание объекта для нормализации
X_train_scaled = scaler.fit_transform(X_train)  # Обучение и преобразование обучающего набора данных
X_test_scaled = scaler.transform(X_test)  # Преобразование тестового набора данных


In [6]:
param_grid = {
    'hidden_layer_sizes': [
        (4,), (5,), (6,)
    ],
   
    'activation': ['logistic'],
    'solver': ['adam'],
    'beta_1': [0.85, 0.9, 0.95],
    'beta_2': [0.99, 0.998],
    'epsilon': [1e-9],
    'alpha': [0.25, 0.3, 0.35],
    'learning_rate_init': [0.03, 0.05, 0.07],
    'early_stopping': [True],
    'n_iter_no_change': [10],
    'tol': [1e-4]
}

# Настройка кросс-валидации
cv = ShuffleSplit(
    n_splits=5,
    test_size=0.25,
    random_state=73
)


grid_search = GridSearchCV(
    estimator=MLPRegressor(random_state=73),
    param_grid=param_grid,
    cv=cv,
    scoring='neg_mean_absolute_percentage_error',
    n_jobs=-1,
    verbose=2
)


In [7]:
# Блок 6: Обучение GridSearchCV
print("Начинается подбор гиперпараметров...")
grid_search.fit(X_train_scaled, y_train)  # Обучение модели с подбором гиперпараметров

Начинается подбор гиперпараметров...
Fitting 5 folds for each of 162 candidates, totalling 810 fits


c:\Users\Серж\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=73, test_size=0.25, train_size=None),
             estimator=MLPRegressor(random_state=73), n_jobs=-1,
             param_grid={'activation': ['logistic'], 'alpha': [0.25, 0.3, 0.35],
                         'beta_1': [0.85, 0.9, 0.95], 'beta_2': [0.99, 0.998],
                         'early_stopping': [True], 'epsilon': [1e-09],
                         'hidden_layer_sizes': [(4,), (5,), (6,)],
                         'learning_rate_init': [0.03, 0.05, 0.07],
                         'n_iter_no_change': [10], 'solver': ['adam'],
                         'tol': [0.0001]},
             scoring='neg_mean_absolute_percentage_error', verbose=2)

In [8]:
# Блок 7: Лучшая модель
best_params = grid_search.best_params_  # Получение лучших гиперпараметров

# Получение модели с лучшими параметрами
best_model = grid_search.best_estimator_

# Прогнозирование на тестовом наборе данных с лучшей моделью
y_pred_best = best_model.predict(X_test_scaled)

In [9]:
# Блок 8: Вычисление метрик
# Вычисление MAPE (Mean Absolute Percentage Error) для оценки точности
mape_best = mean_absolute_percentage_error(y_test, y_pred_best) * 100
# Вычисление RMSE (Root Mean Squared Error) для оценки отклонений предсказаний от реальных значений
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))

print("Лучшие параметры:", grid_search.best_params_)
print("Обучающая выборка:")
print(f"MAPE: {-grid_search.best_score_*100} %")
# Вывод лучших метрик
print("Тестовая выборка:")
print(f"MAPE: {mape_best:.2f}%")
print(f"RMSE: {rmse_best:.2f}")

Лучшие параметры: {'activation': 'logistic', 'alpha': 0.25, 'beta_1': 0.95, 'beta_2': 0.998, 'early_stopping': True, 'epsilon': 1e-09, 'hidden_layer_sizes': (5,), 'learning_rate_init': 0.03, 'n_iter_no_change': 10, 'solver': 'adam', 'tol': 0.0001}
Обучающая выборка:
MAPE: 9.266606051580835 %
Тестовая выборка:
MAPE: 9.28%
RMSE: 2.99


In [10]:
# Блок 9: Создание DataFrame для тестовых значений
# Сравнение истинных значений с предсказанными для тестового набора
test_comparison_model = pd.DataFrame({"yв": y_test.values, "y_pred_best": y_pred_best})
print(test_comparison_model)

    yв  y_pred_best
0   27    24.783380
1   24    24.354603
2   26    25.074849
3   22    22.669064
4   20    22.247489
5   18    22.221057
6   25    23.393709
7   29    23.269406
8   22    23.869652
9   30    22.259093
10  22    22.127257
11  21    18.443772
12  28    25.456071
13  24    24.165881
14  25    24.897029
15  22    25.169307
16  23    24.110450
17  22    24.936680


In [11]:
# Блок 10: Сохранение результатов
# Сохранение результатов и метрик в Excel файл
test_comparison_best = pd.DataFrame({"ус": y_test.values, "y_pred_py": y_pred_best})
metrics_best = pd.DataFrame({"Метрика": ["MAPE", "RMSE"], "Значение": [mape_best, rmse_best]})

# Запись в Excel
output_file = 'models_results_optimized_ус.xlsx'  # Имя выходного файла
with pd.ExcelWriter(output_file) as writer:
    test_comparison_best.to_excel(writer, sheet_name='Результаты лучшей модели', index=False)  # Результаты модели
    metrics_best.to_excel(writer, sheet_name='Метрики лучшей модели', index=False)  # Метрики модели